In [ ]:
#!/usr/bin/env python3
"""
gtt-order-dispatcher.ipynb  –  Zerodha Kite Connect GTT bulk tool
---------------------------------------------------
NEW:  delete **all** GTT orders for a hard-coded symbol
      without using command-line arguments.

HOW-TO
------
1.  Set the three new caps below:
        DELETE_ALL_MODE  = True
        DELETE_SYMBOL    = "NIFTYBEES"
        DELETE_EXCHANGE  = "NSE"
2.  Run:  python3 gtt-order-dispatcher.ipynb
3.  When finished, set DELETE_ALL_MODE back to False
    to resume normal CSV-driven workflow.
"""

import csv
import json
import os
import sys
import time
import webbrowser
import requests
from urllib.parse import urlparse, parse_qs
from kiteconnect import KiteConnect

# ------------------------------------------------------------------
#  API CONFIG CONSTANTS
# ------------------------------------------------------------------
THROTTLE_SEC = 1.0
KITE_API_VERSION_HEADER = "3"
KITE_API_BASE = "https://api.kite.trade"

# ------------------------------------------------------------------
#  USER-EDITABLE CONSTANTS  (only things you ever touch)
# ------------------------------------------------------------------
CSV_PATH = "bulk-gtt-orders.csv"
REPORT_PATH = "gtt-order-placement-report.csv"

# BULK ORDERS DELETE CONTROLS
DELETE_ALL_MODE = False  # <<<< toggle here
DELETE_SYMBOL = "NIFTYBEES"  # <<<< symbol to nuke
DELETE_EXCHANGE = "NSE"  # <<<< exchange for that symbol
# ------------------------------------------------------------------


def get_credentials():
    api_key = os.environ.get("KITE_API_KEY")
    api_secret = os.environ.get("KITE_API_SECRET")
    redirect_url = os.environ.get("KITE_REDIRECT_URL")

    if not api_key:
        api_key = input("Enter your Kite API Key: ").strip()
    if not api_secret:
        api_secret = input("Enter your Kite API Secret: ").strip()
    if not redirect_url:
        redirect_url = input("Enter your Redirect URL: ").strip()
    return api_key, api_secret, redirect_url


def oauth_and_get_tokens(api_key, api_secret, redirect_url):
    kite = KiteConnect(api_key=api_key)
    login_url = kite.login_url()
    print("\nOpening Kite login URL in your browser...")
    print(
        "After logging in, copy the FULL redirect URL from your browser and paste it here.\n"
    )

    webbrowser.open(login_url)
    redirect_input = input("Paste the FULL redirect URL here: ").strip()

    parsed = urlparse(redirect_input)
    qs = parse_qs(parsed.query)
    request_token = (qs.get("request_token") or qs.get("requestToken") or [None])[0]
    if not request_token:
        sys.exit("❌ request_token not found in URL")

    session = kite.generate_session(request_token, api_secret=api_secret)
    kite.set_access_token(session["access_token"])
    print("✅ Access token set\n")
    return kite, session["access_token"]


def headers(api_key, access_token):
    return {
        "X-Kite-Version": KITE_API_VERSION_HEADER,
        "Authorization": f"token {api_key}:{access_token}",
        "Content-Type": "application/x-www-form-urlencoded",
    }


def gtt_triggers_list(h):
    """Return raw list from /gtt/triggers"""
    r = requests.get(f"{KITE_API_BASE}/gtt/triggers", headers=h, timeout=15)
    r.raise_for_status()
    data = r.json()
    return data["data"] if data.get("status") == "success" else []


def delete_gtt_by_id(h, trigger_id):
    url = f"{KITE_API_BASE}/gtt/triggers/{trigger_id}"
    r = requests.delete(url, headers=h, timeout=15)
    r.raise_for_status()  # optional: fail fast on HTTP errors
    return r.json()  # always return the dict


def delete_all_gtt_for_symbol(h, exchange, tradingsymbol):
    """Delete every GTT whose condition matches (exchange, tradingsymbol)"""
    triggers = gtt_triggers_list(h)
    hits = [
        t
        for t in triggers
        if t.get("condition", {}).get("exchange") == exchange
        and t.get("condition", {}).get("tradingsymbol") == tradingsymbol
    ]

    if not hits:
        print(f"ℹ️  No GTT orders found for {tradingsymbol} on {exchange}")
        return []

    print(f"🗑️  Found {len(hits)} GTT(s) – deleting …")
    report = []
    for t in hits:
        tid = t["id"]
        try:
            resp = delete_gtt_by_id(h, tid)
            status = "success" if resp.get("status") == "success" else "failed"
            msg = resp.get("data", resp)
            report.append(
                {
                    "action": "delete_all_by_symbol",
                    "status": status,
                    "message": msg,
                    "trigger_id": tid,
                    "tradingsymbol": tradingsymbol,
                    "exchange": exchange,
                }
            )
            print(f"  ✅ {tid}" if status == "success" else f"  ❌ {tid}  {msg}")
        except Exception as e:
            report.append(
                {
                    "action": "delete_all_by_symbol",
                    "status": "error",
                    "message": str(e),
                    "trigger_id": tid,
                    "tradingsymbol": tradingsymbol,
                    "exchange": exchange,
                }
            )
            print(f"  ⚠️  {tid}  {e}")
        time.sleep(THROTTLE_SEC)
    print("🎯  Bulk-delete complete\n")
    return report


def write_report(path, rows):
    if not rows:
        return
    keys = rows[0].keys()
    with open(path, "w", newline="", encoding="utf-8") as f:
        csv.DictWriter(f, fieldnames=keys).writeheader()
        csv.DictWriter(f, fieldnames=keys).writerows(rows)
    print(f"📄 Report → {path}\n")


# ------------------------------------------------------------------
#  Original CSV workflow helpers
# ------------------------------------------------------------------
def read_csv_rows(csv_path):
    rows = []
    with open(csv_path, newline="", encoding="utf-8") as f:
        for r in csv.DictReader(f):
            rows.append(
                {k: (v.strip() if isinstance(v, str) else v) for k, v in r.items()}
            )
    return rows


def create_single_gtt(h, row):
    url = f"{KITE_API_BASE}/gtt/triggers"
    condition = {
        "exchange": row["exchange"],
        "tradingsymbol": row["tradingsymbol"],
        "trigger_values": [float(row["trigger_price"])],
        "last_price": float(row.get("last_price", 0)),
    }
    order = {
        "exchange": row["exchange"],
        "tradingsymbol": row["tradingsymbol"],
        "transaction_type": row["transaction_type"],
        "quantity": int(float(row["quantity"])),
        "order_type": row.get("order_type", "LIMIT"),
        "product": row.get("product", "CNC"),
        "price": float(row["order_price"]),
    }
    payload = {
        "type": "single",
        "condition": json.dumps(condition),
        "orders": json.dumps([order]),
    }
    return requests.post(url, data=payload, headers=h, timeout=15)


def modify_gtt_by_id(h, trigger_id, row):
    url = f"{KITE_API_BASE}/gtt/triggers/{trigger_id}"
    condition = {
        "exchange": row["exchange"],
        "tradingsymbol": row["tradingsymbol"],
        "trigger_values": [float(row["trigger_price"])],
        "last_price": float(row.get("last_price", 0)),
    }
    order = {
        "exchange": row["exchange"],
        "tradingsymbol": row["tradingsymbol"],
        "transaction_type": row["transaction_type"],
        "quantity": int(float(row["quantity"])),
        "order_type": row.get("order_type", "LIMIT"),
        "product": row.get("product", "CNC"),
        "price": float(row["order_price"]),
    }
    payload = {
        "type": "single",
        "condition": json.dumps(condition),
        "orders": json.dumps([order]),
    }
    return requests.put(url, data=payload, headers=h, timeout=15)


def find_triggers_by_instrument(h, exchange, tradingsymbol, transaction_type=None):
    triggers = gtt_triggers_list(h)
    matches = []
    for t in triggers:
        cond = t.get("condition", {})
        if (
            cond.get("exchange") == exchange
            and cond.get("tradingsymbol") == tradingsymbol
        ):
            if transaction_type:
                orders = t.get("orders", [])
                if not any(
                    o.get("transaction_type") == transaction_type for o in orders
                ):
                    continue
            matches.append(t)
    return matches


def process_csv_rows(kite, api_key, access_token, rows):
    h = headers(api_key, access_token)
    report = []
    for idx, row in enumerate(rows, start=1):
        action = (row.get("action") or "").lower()
        gtt_id = row.get("gtt_id", "").strip()
        exchange = row["exchange"]
        tradingsymbol = row["tradingsymbol"]
        transaction_type = row.get("transaction_type")

        try:
            if action == "create":
                r = create_single_gtt(h, row)
            elif action == "modify":
                if not gtt_id:
                    matches = find_triggers_by_instrument(
                        h, exchange, tradingsymbol, transaction_type
                    )
                    if not matches:
                        raise ValueError("No matching GTT found to modify.")
                    gtt_id = matches[0]["id"]
                r = modify_gtt_by_id(h, gtt_id, row)
            elif action == "delete":
                if not gtt_id:
                    matches = find_triggers_by_instrument(
                        h, exchange, tradingsymbol, transaction_type
                    )
                    if not matches:
                        raise ValueError("No matching GTT found to delete.")
                    gtt_id = matches[0]["id"]
                r = delete_gtt_by_id(h, gtt_id)
            else:
                report.append(
                    {
                        "row_index": idx,
                        "action": action,
                        "status": "skipped",
                        "message": "Unknown action",
                        "trigger_id": "",
                    }
                )
                continue

            data = r.json()
            status = "success" if data.get("status") == "success" else "failed"
            msg = data.get("data", data)
            trigger_id = (
                (msg.get("trigger_id") or msg.get("id"))
                if isinstance(msg, dict)
                else ""
            )
            report.append(
                {
                    "row_index": idx,
                    "action": action,
                    "status": status,
                    "message": msg,
                    "trigger_id": trigger_id,
                }
            )
        except Exception as e:
            report.append(
                {
                    "row_index": idx,
                    "action": action,
                    "status": "error",
                    "message": str(e),
                    "trigger_id": "",
                }
            )
        time.sleep(THROTTLE_SEC)
    return report


# ------------------------------------------------------------------
#  Main entry-point
# ------------------------------------------------------------------
def main():
    print("🔁 Bulk GTT Automation Script for Zerodha Kite\n")

    api_key, api_secret, redirect_url = get_credentials()
    kite, access_token = oauth_and_get_tokens(api_key, api_secret, redirect_url)

    if DELETE_ALL_MODE:
        h = headers(api_key, access_token)
        rep = delete_all_gtt_for_symbol(h, DELETE_EXCHANGE, DELETE_SYMBOL)
        if rep:
            write_report(f"gtt-delete-all-{DELETE_SYMBOL}.csv", rep)
        return

    if not os.path.exists(CSV_PATH):
        sys.exit(f"❌ CSV file not found: {CSV_PATH}")

    rows = read_csv_rows(CSV_PATH)
    print(f"Processing {len(rows)} rows from {CSV_PATH}…\n")
    rep = process_csv_rows(kite, api_key, access_token, rows)
    write_report(REPORT_PATH, rep)
    print("✅ All done.\n")


if __name__ == "__main__":
    main()

🔁 Bulk GTT Automation Script for Zerodha Kite


Opening Kite login URL …
✅ Access token set

🗑️  Found 2 GTT(s) – deleting …
  ✅ 292754000
  ✅ 292753990
🎯  Bulk-delete complete

📄 Report → gtt-delete-all-SMALLCAP.csv

